# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117616e+02     1.676715e+00
 * time: 0.4563889503479004
     1     1.124772e+01     8.874551e-01
 * time: 1.0324938297271729
     2    -1.097662e+01     8.995013e-01
 * time: 1.1024978160858154
     3    -3.386093e+01     6.823536e-01
 * time: 1.221038818359375
     4    -4.723202e+01     5.038886e-01
 * time: 1.3034520149230957
     5    -5.685737e+01     1.930724e-01
 * time: 1.39107084274292
     6    -5.975275e+01     1.221700e-01
 * time: 1.4613468647003174
     7    -6.087395e+01     4.711003e-02
 * time: 1.5317199230194092
     8    -6.131357e+01     4.786881e-02
 * time: 1.676400899887085
     9    -6.157330e+01     3.761243e-02
 * time: 1.738804817199707
    10    -6.179191e+01     2.862968e-02
 * time: 1.8017148971557617
    11    -6.195419e+01     1.862270e-02
 * time: 1.8644869327545166
    12    -6.202980e+01     1.495031e-02
 * time: 1.9270789623260498
    13    -6.209433e+01     1.636484e-02
 * time: 2.328671932220459

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671061
    AtomicLocal         -18.8557636
    AtomicNonlocal      14.8522616
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485364 
    Xc                  -19.3336818

    total               -62.261666459411